In [17]:
import pandas as pd
import numpy as np

In [2]:
import sentence_transformers as st

In [3]:
model = st.SentenceTransformer('distilbert-base-nli-mean-tokens')

In [4]:
model.encode(["Machine Learning Tokyo is an awesome community. By the way, I like mangos."])

[array([-1.18456936e+00, -3.80751252e-01,  5.70952177e-01, -1.96907520e-01,
        -1.24814343e+00, -2.35962942e-01,  2.62315363e-01, -3.50492865e-01,
         2.60189295e-01,  1.36031747e-01,  3.52496415e-01,  8.33485484e-01,
        -7.96369731e-01,  1.02349150e+00, -3.06178570e-01, -1.26091659e+00,
         7.94607222e-01,  9.70712826e-02, -1.76925182e-01, -1.48145616e-01,
         1.09787977e+00, -1.45094365e-01, -5.03086984e-01,  8.69092643e-01,
        -5.20024776e-01, -2.81262040e-01,  9.72246587e-01,  1.09444082e-01,
         2.16068193e-01,  6.49353385e-01, -9.38707143e-02,  8.99284184e-01,
        -1.15091302e-01, -1.13751963e-01,  1.36130139e-01, -2.01750323e-01,
         2.38336459e-01, -2.62810469e-01,  2.92719632e-01, -4.20371175e-01,
         4.04634356e-01, -9.92321908e-01,  5.55080287e-02,  1.06900550e-01,
         4.30007428e-01, -3.32337141e-01, -9.34900105e-01,  4.11898047e-01,
        -1.16922259e+00,  3.00250016e-02,  2.46032014e-01,  2.16064170e-01,
         3.4

In [5]:
from twitterscraper import query_tweets, query_tweets_from_user

In [6]:
from twitterscraper.query import query_single_page

In [10]:
def query_tweets_from_user_custom(user, lang, limit=40):
    pos=None
    tweets = []
    while True:
        new_tweets, pos = query_single_page(user, lang=lang, pos=pos, from_user=True, use_proxy=True)
        if len(new_tweets) == 0:
            return tweets
        tweets += new_tweets
        if len(tweets) >= limit:
            return tweets

In [61]:
%%time
results = []
name_dict = dict()
for screen_name in ['peacej','suzatweet','geoffreyhinton','kaifulee','GoogleAI','AndrewYNg','OpenAI',
                    'kdnuggets','ylecun','lexfridman','facebookai','drfeifei','elonmusk','sama','goodfellow_ian',
                    'TensorFlow','PyTorch','SebastianThrun','NumFOCUS','hardmaru','AllenNLP','JeffDean','OpenAI','huggingface',
                    'chrmanning','DeepMind','GoogleBrain','__MLT__','rheza_h','fchollet'
                   ]:
    for tweet in query_tweets_from_user_custom(screen_name,limit=40,lang='en'):
        if screen_name not in name_dict:
            if screen_name == tweet.screen_name:
                name_dict[screen_name] = tweet.username # if not equal, probably because it's a retweet
            else:
                continue
        text = tweet.text
        for link in tweet.links:
            text = text.replace(link,'')
        if len(text) < 5:
            continue
        results.append([screen_name,name_dict[screen_name],text])

In [62]:
df = pd.DataFrame(results, columns=['screen_name','user_name','text'])

In [63]:
len(df)

1289

In [64]:
%%time
df['embedding'] = model.encode(df['text'].tolist())

CPU times: user 4min 15s, sys: 13.1 s, total: 4min 28s
Wall time: 50.9 s


In [65]:
df['user_name'] = df['user_name'].apply(lambda x: x.replace('ć','c')) # tensorboard visualization can't handle special characters?

In [66]:
df.head(5)

,screen_name,user_name,text,embedding
0,peacej,Jerry Chi,Spark Meetup Tokyo #3 Onlineに参加を申し込みました！ … #s...,"[-0.37950185, -0.6813484, 0.4439314, -0.868268..."
1,peacej,Jerry Chi,I'll be giving an online technical talk about ...,"[0.21104087, -0.030900031, 0.67631316, -0.5976..."
2,peacej,Jerry Chi,"I have a joke about Gaussian processes, but yo...","[-0.049494036, -0.27678245, 1.2959975, -0.1334..."
3,peacej,Jerry Chi,We're hiring in Tokyo! (considerable ML experi...,"[-0.5225698, -0.59153587, 0.49577758, -1.04341..."
4,peacej,Jerry Chi,I'm happy this little-known data scientist hel...,"[-0.357002, -0.23999633, -0.1265356, -0.665570..."


In [67]:
def elementwise_avg(arr):
    return np.mean(np.array(arr), axis=0).tolist()

In [68]:
dfagg = df.groupby('user_name').aggregate({'embedding':elementwise_avg}).reset_index()

In [69]:
dfagg.head(20)

,user_name,embedding
0,Andrew Ng,"[-0.17855486273765564, -0.22813737392425537, 0..."
1,Christopher Manning,"[-0.3423832952976227, -0.20097926259040833, 0...."
2,DeepMind,"[-0.24083568155765533, -0.14192964136600494, 0..."
3,Elon Musk,"[-0.3248625099658966, -0.26285520195961, 0.471..."
4,Facebook AI,"[-0.19221384823322296, -0.3242705464363098, 0...."
5,Fei-Fei Li,"[-0.20383456349372864, -0.19125691056251526, 0..."
6,François Chollet,"[-0.3811887502670288, -0.31465452909469604, 0...."
7,Geoffrey Hinton,"[-0.3778385818004608, -0.30991941690444946, 0...."
8,Google AI,"[-0.4932260811328888, -0.18140526115894318, 0...."
9,Hugging Face,"[-0.37294384837150574, -0.29550179839134216, 0..."


In [1]:
import tensorflow as tf
import tensorboard as tb
import datetime, os
from torch.utils.tensorboard import SummaryWriter
tf.io.gfile = tb.compat.tensorflow_stub.io.gfile

In [37]:
%load_ext tensorboard

In [70]:
vectors = np.array([np.array(x) for x in dfagg['embedding']])
metadata = dfagg['user_name'].tolist()
writer = SummaryWriter()
writer.add_embedding(vectors, metadata)
writer.close()

In [72]:
%tensorboard --logdir=runs

Reusing TensorBoard on port 6006 (pid 84285), started 0:26:22 ago. (Use '!kill 84285' to kill it.)

In [29]:
#tweet object example

{'screen_name': 'peacej',
 'username': 'Jerry Chi',
 'user_id': '20598022',
 'tweet_id': '1288824131618607104',
 'tweet_url': '/peacej/status/1288824131618607104',
 'timestamp': datetime.datetime(2020, 7, 30, 13, 9, 52),
 'timestamp_epochs': 1596114592,
 'text': 'Spark Meetup Tokyo #3 Onlineに参加を申し込みました！ https://spark-meetup-tokyo.connpass.com/event/181422/?utm_campaign=event_participate_to_follower&utm_source=notifications&utm_medium=twitter\xa0… #sparktokyo',
 'text_html': '<p class="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text" data-aria-label-part="0" lang="ja">Spark Meetup Tokyo #3 Onlineに参加を申し込みました！ <a class="twitter-timeline-link" data-expanded-url="https://spark-meetup-tokyo.connpass.com/event/181422/?utm_campaign=event_participate_to_follower&amp;utm_source=notifications&amp;utm_medium=twitter" dir="ltr" href="https://t.co/upwzeSK5Zy" rel="nofollow noopener" target="_blank" title="https://spark-meetup-tokyo.connpass.com/event/181422/?utm_campaign=event_participa